In [3]:
!pip install torch_geometric rdkit mols2grid --quiet

In [4]:
from rdkit import Chem

import torch
import torch.nn.functional as F
from torch.nn import GRU, Linear, ReLU, Sequential

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, Set2Set
from torch_geometric.utils import remove_self_loops
import os

In [5]:
target = 2

class MyTransform(object):
    def __call__(self, data):
        # Specify target.
        data.y = data.y[:, target]
        return data


class Complete(object):
    def __call__(self, data):
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data

In [6]:
transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
dataset = QM9(root='qm9',transform=transform).shuffle()

# Normalize targets to mean = 0 and std = 1.
mean = dataset.data.y.mean(dim=0, keepdim=True)
std = dataset.data.y.std(dim=0, keepdim=True)
dataset.data.y = (dataset.data.y - mean) / std
mean, std = mean[:, target].item(), std[:, target].item()

train_test_split = 0.1
split = int((1-train_test_split)*len(dataset))

train_loader = DataLoader(dataset[:split], batch_size=128, shuffle=True)
test_loader = DataLoader(dataset[split:], batch_size=128, shuffle=False)

/home/jielong/.local/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [7]:
class Net(torch.nn.Module):
    def __init__(self,dim):
        super().__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
        return out.view(-1)

model = Net(dim=64)
print(model)

Net(
  (lin0): Linear(in_features=11, out_features=64, bias=True)
  (conv): NNConv(64, 64, aggr=mean, nn=Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=4096, bias=True)
  ))
  (gru): GRU(64, 64)
  (set2set): Set2Set(64, 128)
  (lin1): Linear(in_features=128, out_features=64, bias=True)
  (lin2): Linear(in_features=64, out_features=1, bias=True)
)


In [8]:
from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001)
os.makedirs("output",exist_ok=True)
def train(epoch):
    model.train()
    loss_all = 0
    
    pbar = tqdm(train_loader)
    for data in pbar:
        data = data.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(model(data), data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        pbar.set_postfix(epoch=epoch, loss=loss.item())
    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
    error = 0

    for data in tqdm(loader):
        data = data.to(device)
        error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
    return error / len(loader.dataset)

best_val_error = None
for epoch in range(1, 201):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    test_error = test(test_loader)
    scheduler.step(test_error)

    print(f'Epoch: {epoch:03d}, LR: {lr:7f}, Loss: {loss:.7f}, Test MAE: {test_error:.7f}')

    if epoch%10==0:
        torch.save(model, f"output/model_epoch{epoch}.pt")

  0%|          | 0/920 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

Epoch: 001, LR: 0.001000, Loss: 0.5226028, Test MAE: 0.2654813


  0%|          | 0/920 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

...

  0%|          | 0/103 [00:00<?, ?it/s]

Epoch: 199, LR: 0.000010, Loss: 0.0035361, Test MAE: 0.0556545


  0%|          | 0/920 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

Epoch: 200, LR: 0.000010, Loss: 0.0035364, Test MAE: 0.0556213
